In [1]:
import pandas as pd
import os, time

# determine 涨停
def isZhangting(s_df, index=-1):
    if (s_df.iloc[index].close/s_df.iloc[index-1].close) >= 1.099:
        return True
    else:
        return False

#determine 上涨
def isShangzhang(s_df, index=-1):
    if len(s_df.index) >= (index)+1:
        return ((s_df.iloc[index].close/s_df.iloc[index-1].close-1) > 0.02)
    else:
        return False

#determine 条件一
def isQualifiedCondition1(s_df, index=-1):
    if len(stock_df.index)>=(abs(index)+2):
        if (s_df.iloc[index].close * s_df.iloc[index].volume > 200000):
            return  (isZhangting(s_df, index-1) and isShangzhang(s_df,index))
    return False

#determine 条件二
def isQualifiedCondition2(s_df, index=-1):
    if len(stock_df.index)>=(abs(index)+3):
        if (s_df.iloc[index].close / s_df.iloc[index-2].close < 0.92) \
                and (s_df.iloc[index].close<s_df.iloc[index-1].close) \
                and (s_df.iloc[index-1].close<s_df.iloc[index-2].close)\
                and isZhangting(s_df, index-2):
            return True
    return False

#determine 条件三
def isQualifiedCondition3(s_df, index=-1):
    if len(stock_df.index)>=(abs(index)+3): 
        if isQualifiedCondition1(s_df, index-1): 
            if (s_df.iloc[index].close / s_df.iloc[index-1].close < 0.95):
                return True
        return False
        
def completeCode(code_str):
    if len(code_str)<6:
        return "0"*(6-len(code_str))+code_str
    else:
        return code_str

print (time.strftime("%M:%S"))
    
########################################
datafolder_dir = os.getcwd()+"/20171013"
########################################

file_list = os.listdir(datafolder_dir)

file_index = 0
select_list = []
file_dict = {}
for file in file_list:
    #print ("Working on file %s: %s"%(file_index, file))
    file_index = file_index + 1
    if ("day" in file):
        stock_df = pd.read_csv(datafolder_dir+"/"+file)
        if isQualifiedCondition1(stock_df):
            select_list.append("condition 1:" + completeCode(str(stock_df.iloc[0].code)))
            file_dict[completeCode(str(stock_df.iloc[0].code))] = datafolder_dir+"/"+file
        if isQualifiedCondition2(stock_df):
            select_list.append("condition 2:" + completeCode(str(stock_df.iloc[0].code)))
            file_dict[completeCode(str(stock_df.iloc[0].code))] = datafolder_dir+"/"+file
        if isQualifiedCondition3(stock_df):
            select_list.append("condition 3:" + completeCode(str(stock_df.iloc[0].code)))
            file_dict[completeCode(str(stock_df.iloc[0].code))] = datafolder_dir+"/"+file

select_list.sort()

for file in select_list:
    print (file)

print (file_dict)
print (time.strftime("%M:%S"))


41:11
condition 1:000791
condition 1:002017
condition 1:002076
condition 1:002194
condition 1:002264
condition 1:002432
condition 1:002843
condition 1:002856
condition 1:002893
condition 1:300277
condition 1:300562
condition 1:300565
condition 1:300585
condition 1:300597
condition 1:300700
condition 1:603136
condition 1:603157
condition 1:603378
condition 1:603813
condition 1:603888
condition 1:603963
condition 2:002813
condition 2:300556
condition 3:000301
condition 3:002662
condition 3:603501
{'603813': '/root/notebooks/tus/20171013/day_603813.csv', '002194': '/root/notebooks/tus/20171013/day_002194.csv', '002432': '/root/notebooks/tus/20171013/day_002432.csv', '603136': '/root/notebooks/tus/20171013/day_603136.csv', '002843': '/root/notebooks/tus/20171013/day_002843.csv', '603501': '/root/notebooks/tus/20171013/day_603501.csv', '002076': '/root/notebooks/tus/20171013/day_002076.csv', '002813': '/root/notebooks/tus/20171013/day_002813.csv', '603157': '/root/notebooks/tus/20171013/day

In [2]:
for index in range(len(select_list)):
    select_list[index] = select_list[index][-6:]

print (select_list)

['000791', '002017', '002076', '002194', '002264', '002432', '002843', '002856', '002893', '300277', '300562', '300565', '300585', '300597', '300700', '603136', '603157', '603378', '603813', '603888', '603963', '002813', '300556', '000301', '002662', '603501']


In [3]:
stock_df_list = []
for item in select_list:
    print (item)
    stock_df_list.append(pd.read_csv(file_dict[item]))

print (len(stock_df_list))
print (stock_df_list[1].tail())

000791
002017
002076
002194
002264
002432
002843
002856
002893
300277
300562
300565
300585
300597
300700
603136
603157
603378
603813
603888
603963
002813
300556
000301
002662
603501
26
           date   open  close   high    low    volume  code
636  2017-10-09  11.85  11.77  12.09  11.60  169779.0  2017
637  2017-10-10  11.74  11.87  11.87  11.47  112729.0  2017
638  2017-10-11  11.72  11.40  11.77  11.35   84621.0  2017
639  2017-10-12  11.40  12.54  12.54  11.40  165242.0  2017
640  2017-10-13  12.87  13.05  13.38  12.66  402844.0  2017


In [4]:
import tushare as ts
for index in range(len(select_list)):
    df = ts.get_realtime_quotes(select_list[index])
    # print (float(stock_df_list[index].tail(1).close), float(df.iloc[0]["price"]))
    ratio = (float(df.iloc[0]["price"]) / float(stock_df_list[index].tail(1).close))
    if ratio < 0.97 and ratio > 0.85:
        print (select_list[index], "diff is %s"%((1-ratio)*100))